# Customer Clustering: Multi-Perspective Approach

This notebook identifies customer clusters from different perspectives (behavioral, profile), compares clustering algorithms, and merges the best solutions using hierarchical clustering on centroids.

In [ ]:
# All required imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import DBSCAN, KMeans, MeanShift, estimate_bandwidth
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture
from collections import Counter

# Preprocessing & Feature Engineering

All custom functions for preprocessing, feature engineering, and aggregation are defined below.

In [ ]:
def winsorize_dataframe(df, columns, limits=(0.01, 0.01)):
    """
    Apply winsorization to each column in `columns`.
    limits=(lower_pct, upper_pct) means: cap values at the 1st and 99th percentile.

    Returns the winsorized copy of df.
    """
    df = df.copy()
    for col in columns:
        if col in df.columns:
            # winsorize returns masked arrays -> convert to normal array
            df[col] = winsorize(df[col], limits=limits).data
    return df

def preprocess_flights(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the FlightsDB:
    - Winsorize outliers
    - Convert YearMonthDate to datetime
    - Round down NumFlights and NumFlightsWithCompanions
    - Set DistanceKM = 0 where NumFlights == 0
    - Drop DollarCostPointsRedeemed
    - Add log-transformed versions of skewed variables
    - Create PointsUtilizationRatio = PointsRedeemed / PointsAccumulated
    """
    df = flights_df.copy()

    # 0. Winsorize outliers (Flights DB outliers are legitimate but skewed)
    outlier_cols = [
        'NumFlights', 'NumFlightsWithCompanions', 'DistanceKM', 
        'PointsAccumulated', 'PointsRedeemed'
    ]
    df = winsorize_dataframe(df, outlier_cols, limits=(0.01, 0.01))

    # 1. YearMonthDate -> datetime
    if 'YearMonthDate' in df.columns:
        df['YearMonthDate'] = pd.to_datetime(df['YearMonthDate'])

    # 2. Round down flight counts and cast to int
    for col in ['NumFlights', 'NumFlightsWithCompanions']:
        if col in df.columns:
            df[col] = np.floor(df[col]).astype(int)

    # 3. Fix logical inconsistency: DistanceKM must be 0 if NumFlights == 0
    if {'NumFlights', 'DistanceKM'}.issubset(df.columns):
        df.loc[df['NumFlights'] == 0, 'DistanceKM'] = 0

    # 4. Drop perfectly correlated variable
    if 'DollarCostPointsRedeemed' in df.columns:
        df = df.drop(columns=['DollarCostPointsRedeemed'])

    # 5. Log transforms for skewed numeric variables
    log_cols = ['DistanceKM', 'PointsAccumulated', 'PointsRedeemed', 'NumFlights']
    for col in log_cols:
        if col in df.columns:
            df[f'{col}_log'] = np.log1p(df[col])

    # 6. Points utilisation ratio
    if {'PointsRedeemed', 'PointsAccumulated'}.issubset(df.columns):
        denom = df['PointsAccumulated'].replace({0: np.nan})
        df['PointsUtilizationRatio'] = df['PointsRedeemed'] / denom

    return df

def preprocess_customers(customer_df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply preprocessing steps to the CustomerDB:
    - Create cancellation flag from CancellationDate
    - Group-median imputation (by LoyaltyStatus) for Income and Customer Lifetime Value
    - Winsorize outliers (Income, CLV)
    - Log transform Customer Lifetime Value and Income
    - Encode Gender as binary
    """
    df = customer_df.copy()

    # 1. Cancellation flag
    if 'CancellationDate' in df.columns:
        df['CancelledFlag'] = df['CancellationDate'].notna().astype(int)

    # 2. Group-median imputation by LoyaltyStatus
    group_col = 'LoyaltyStatus'
    cols_to_impute = ['Income', 'Customer Lifetime Value']
    for col in cols_to_impute:
        if col in df.columns and group_col in df.columns:
            df[col] = df.groupby(group_col)[col].transform(
                lambda x: x.fillna(x.median())
            )

    # 3. Winsorize outliers
    outlier_cols = ['Income', 'Customer Lifetime Value']
    df = winsorize_dataframe(df, outlier_cols, limits=(0.01, 0.01))

    # 4. Log transforms
    if 'Customer Lifetime Value' in df.columns:
        df['CLV_log'] = np.log1p(df['Customer Lifetime Value'])
    if 'Income' in df.columns:
        df['Income_log'] = np.log1p(df['Income'].clip(lower=0))

    # 5. Gender encoding
    if 'Gender' in df.columns:
        df['Gender'] = df['Gender'].map({'female': 1, 'male': 0}).fillna(0).astype(int)

    # 6. Education to Years (Ordinal Encoding)
    if 'Education' in df.columns:
        edu_map = {
            'High School or Below': 12,
            'College': 14,
            'Bachelor': 16,
            'Master': 18,
            'Doctor': 21
        }
        df['Education'] = df['Education'].map(edu_map)
        df['Education'] = df['Education'].fillna(16)

    # 7. Turn marital status into a flag
    if 'Marital Status' in df.columns:
        df['Marital Status'] = np.where(df['Marital Status'] != 'Married', 1, 0)

    # 8. Tenure
    ref_date = pd.to_datetime('2022-01-01')
    if 'EnrollmentDateOpening' in df.columns:
        df['EnrollmentDateOpening'] = pd.to_datetime(df['EnrollmentDateOpening'])
        df['TenureMonths'] = (ref_date - df['EnrollmentDateOpening']) / pd.Timedelta(days=30.44)

    return df

def build_customer_flight_features(flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate monthly flight records into customer-level features:
    - TotalFlights, TotalDistanceKM, TotalPointsAccumulated, TotalPointsRedeemed
    - MeanPointsUtilization
    - AverageFlightDistance
    """
    id_col = 'Loyalty#'
    df = flights_df.copy()
    
    agg = (
        df
        .groupby(id_col)
        .agg(
            TotalFlights=('NumFlights', 'sum'),
            TotalDistanceKM=('DistanceKM', 'sum'),
            TotalPointsAccumulated=('PointsAccumulated', 'sum'),
            TotalPointsRedeemed=('PointsRedeemed', 'sum'),
            MeanPointsUtilization=('PointsUtilizationRatio', 'mean')
        )
        .reset_index()
    )

    # Log transforms for aggregated features
    for col in ['TotalFlights', 'TotalDistanceKM', 'TotalPointsAccumulated', 'TotalPointsRedeemed']:
        agg[f'{col}_log'] = np.log1p(agg[col])
    
    # Average flight distance
    agg['AverageFlightDistance'] = agg['TotalDistanceKM'] / agg['TotalFlights'].replace({0: np.nan})

    return agg

def create_model_df(customer_df: pd.DataFrame, flights_df: pd.DataFrame) -> pd.DataFrame:
    """
    Orchestrates the creation of the final modeling dataframe:
    1. Preprocess customers and flights
    2. Build customer-level flight features
    3. Merge datasets (Left Join)
    4. Set Loyalty# as Index
    5. Handle missing values
    6. Encode categorical variables (OneHotEncoder)
    7. Drop unnecessary columns
    8. Scale numeric features (StandardScaler)
    """
    # 1. Preprocess
    cust_clean = preprocess_customers(customer_df)
    flights_clean = preprocess_flights(flights_df)

    # 2. Build flight features
    flight_features = build_customer_flight_features(flights_clean)

    # 3. Merge
    model_df = cust_clean.merge(flight_features, on='Loyalty#', how='left')

    # 4. Set Loyalty# as Index
    if 'Loyalty#' in model_df.columns:
        model_df.set_index('Loyalty#', inplace=True)

    # 5. Handle Missing Values (Numeric)
    numeric_cols_to_fill = model_df.select_dtypes(include=[np.number]).columns
    model_df[numeric_cols_to_fill] = model_df[numeric_cols_to_fill].fillna(0)

    # 6. Drop unnecessary columns
    cols_to_drop = [
        'First Name', 'Last Name', 'CancellationDate', 'Customer Name',
        'Country', 'Province or State', 'City', 'Postal Code',
        'Latitude', 'Longitude', 'EnrollmentDateOpening', 'EnrollmentType',
        'TotalFlights', 'TotalDistanceKM', 'TotalPointsAccumulated', 'TotalPointsRedeemed',
        'Customer Lifetime Value', 'Income'
    ]
    model_df = model_df.drop(columns=[c for c in cols_to_drop if c in model_df.columns], errors='ignore')

    # 7. Separate Numeric and Categorical
    categorical_cols = ['LoyaltyStatus', 'Location Code']
    categorical_cols = [c for c in categorical_cols if c in model_df.columns]
       
    numeric_cols = model_df.select_dtypes(include=[np.number]).columns.tolist()
    
    # Exclude binary/ordinal from scaling
    unscaled_cols = []
    for col in ['CancelledFlag', 'Marital Status', 'Gender']:
        if col in numeric_cols:
            numeric_cols.remove(col)
            unscaled_cols.append(col)

    # 8. OneHotEncoding
    ohe = OneHotEncoder(sparse_output=False, drop='first', dtype=int)
    encoded_data = ohe.fit_transform(model_df[categorical_cols])
    encoded_cols = ohe.get_feature_names_out(categorical_cols)
    
    df_encoded = pd.DataFrame(encoded_data, columns=encoded_cols, index=model_df.index)
    
    # 9. Scale Numeric Features
    scaler = StandardScaler()
    scaled_numeric = scaler.fit_transform(model_df[numeric_cols])
    df_numeric_scaled = pd.DataFrame(scaled_numeric, columns=numeric_cols, index=model_df.index)
    
    # 10. Combine
    dfs_to_concat = [df_numeric_scaled, df_encoded]
    if unscaled_cols:
        dfs_to_concat.append(model_df[unscaled_cols])
        
    df_final = pd.concat(dfs_to_concat, axis=1)
    
    return df_final

def evaluate_clustering(algorithm_cls, X, param_name, param_values):
    """
    Evaluate clustering algorithm with multiple metrics.
    Prints number of clusters, Silhouette, Davies-Bouldin, and R2 metrics (rounded to 2 decimals).
    """
    algo_name = algorithm_cls.__name__
    print(f"\n{algo_name} Clustering Results:")
    for param in param_values:
        model = algorithm_cls(**{param_name: param})
        labels = model.fit_predict(X)
        n_clusters = len(set(labels))
        sil = silhouette_score(X, labels) if n_clusters > 1 else None
        db = davies_bouldin_score(X, labels) if n_clusters > 1 else None
        # R2 metric: ratio of between-cluster variance to total variance
        if n_clusters > 1:
            cluster_means = np.array([X[labels == k].mean(axis=0) for k in set(labels)])
            overall_mean = X.mean(axis=0)
            between_var = sum([(len(X[labels == k]) * np.sum((cluster_means[i] - overall_mean) ** 2)) for i, k in enumerate(set(labels))])
            total_var = np.sum((X - overall_mean) ** 2)
            r2 = between_var / total_var if total_var > 0 else None
        else:
            r2 = None
        # Round metrics to 2 decimals if not None
        sil_str = f"{sil:.2f}" if sil is not None else "NA"
        db_str = f"{db:.2f}" if db is not None else "NA"
        r2_str = f"{r2:.2f}" if r2 is not None else "NA"
        print(f"{algo_name} ({param_name}={param}): Clusters={n_clusters}, Silhouette={sil_str}, Davies-Bouldin={db_str}, R2={r2_str}")

def apply_pca(X, n_components=0.95):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    print(f"PCA reduced to {X_pca.shape[1]} components, explained variance: {pca.explained_variance_ratio_.sum():.2f}")
    return X_pca

# Data Import

Load customer and flight data from CSV files.

In [ ]:
# Load the data
customer_db = pd.read_csv("data/DM_AIAI_CustomerDB.csv", index_col=0 )
flights_db = pd.read_csv("data/DM_AIAI_FlightsDB.csv")

# Feature Set Definition

Define behavioral and profile feature sets for clustering perspectives.

In [ ]:
# Remove duplicates in customer database
initial_rows = customer_db.shape[0]
duplicated_loyalty_ids = customer_db[customer_db['Loyalty#'].duplicated()]['Loyalty#'].unique()
customer_db = customer_db.drop_duplicates(subset=['Loyalty#'])
dropped_rows = initial_rows - customer_db.shape[0]
dropped_percentage = (dropped_rows / initial_rows) * 100

print(f"Dropped {dropped_rows} duplicate customers ({dropped_percentage:.2f}%).")

In [ ]:
# Create the modeling dataset using the pipeline function
# This handles preprocessing, merging, missing values, encoding, feature selection AND scaling
model_df = create_model_df(customer_db, flights_db)
model_df.head()

In [ ]:
# Define feature categories for profile and behavior segmentation
behavior_features = [
    'TotalFlights_log', 'TotalDistanceKM_log', 'TotalPointsAccumulated_log', 
    'TotalPointsRedeemed_log', 'MeanPointsUtilization', 'AverageFlightDistance', 
]

profile_features = [
    'CLV_log', 'Income_log', 'Gender', 'Education', 'Marital Status',
    'LoyaltyStatus_Nova', 'LoyaltyStatus_Star', 
    'Location Code_Suburban', 'Location Code_Urban', 'TenureMonths',
    'CancelledFlag'
]

print(f"Behavior features: {behavior_features}")
print(f"Profile features: {profile_features}")


In [ ]:
# Check for missing values in the model dataframe
model_df.isnull().sum()


# Correlation Analysis

Analyze and remove highly correlated features to improve clustering quality.

In [ ]:
# Correlation Analysis and Feature Selection
# 1. Plot initial correlation matrix
plt.figure(figsize=(24, 20))
initial_corr = model_df.corr()
sns.heatmap(initial_corr, annot=True, cmap='coolwarm', center=0)
plt.title('Initial Correlation Matrix')
plt.show()

# 2. Identify and remove highly correlated features (> 0.8)
# We use the absolute correlation matrix
corr_matrix = model_df.corr().abs()

# We select the upper triangle of the correlation matrix. 
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# We drop the column (Feature B) if it has a correlation > 0.8 with any previous column (Feature A).
# This way, Feature A is kept and Feature B is removed.
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]

print(f"Dropping highly correlated features: {to_drop}")

# Drop features from model_df
model_df = model_df.drop(columns=to_drop)

# Update feature lists to remove dropped columns
behavior_features = [f for f in behavior_features if f not in to_drop]
profile_features = [f for f in profile_features if f not in to_drop]

print(f"Updated behavior features: {behavior_features}")
print(f"Updated profile features: {profile_features}")

# 3. Plot updated correlation matrix
plt.figure(figsize=(20, 16))
sns.heatmap(model_df.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix After Removing Highly Correlated Features')
plt.show()


# Outlier Detection

Detect and remove multivariate outliers using DBSCAN before clustering.

DBSCAN is applied to the scaled feature space for outlier detection. In this project, DBSCAN is used exclusively for identifying anomalous customers, not for final segmentation.

In [ ]:
# Use model_df as X_scaled (Loyalty# is now the index)
dbscan = DBSCAN(eps=1.9, min_samples=20, n_jobs=1)
dbscan_labels = dbscan.fit_predict(model_df)

outlier_count = Counter(dbscan_labels)
print(f"DBSCAN results: {outlier_count}")
print(f"Outliers detected: {outlier_count.get(-1, 0)}")
print(f"Core customers: {outlier_count.get(0, 0)}")

core_mask = (dbscan_labels != -1)

model_df_clipped = model_df[core_mask]
outliers_df = model_df[dbscan_labels == -1]

print(f"Core customers kept: {len(model_df_clipped):,}")

# Clustering Algorithms Comparison


## Clustering: Multiple Algorithms & Perspectives

We now test several clustering algorithms (KMeans, DBSCAN, MeanShift) with different hyperparameters, This approach allows us to compare cluster quality and interpretability across perspectives and methods.

In [ ]:
# Prepare feature sets for clustering
behavior_df = model_df_clipped[behavior_features]
profile_df = model_df_clipped[profile_features]

# Apply PCA to reduce dimensionality for each feature set

behavior_df_pca = apply_pca(behavior_df.values)
profile_df_pca = apply_pca(profile_df.values)

In [ ]:
# KMeans on behavior features
kmeans_params = [2, 3, 4, 5, 6]
evaluate_clustering(KMeans, behavior_df_pca, 'n_clusters', kmeans_params)

# KMeans on profile features
evaluate_clustering(KMeans, profile_df_pca, 'n_clusters', kmeans_params)

# MeanShift on behavior features
meanshift_bandwidths = [0.5, 1.0, 1.5, 2.0]
evaluate_clustering(MeanShift, behavior_df_pca, 'bandwidth', meanshift_bandwidths)

# MeanShift on profile features
evaluate_clustering(MeanShift, profile_df_pca, 'bandwidth', meanshift_bandwidths)


# Next Steps: Merge Cluster Solutions

After identifying the best clustering solutions for each perspective, merge them using hierarchical clustering on the centroids.